In [ ]:
# MuJoCo, 강화학습 툴, 비디오 저장용 라이브러리 설치

# 1. 시스템 라이브러리 설치 (화면 렌더링에 필요한 드라이버 강제 설치)
!apt-get update && apt-get install -y libgl1-mesa-dev libgl1-mesa-glx libglew-osmesa0 libgl1-mesa-dri

# 2. 환경 변수 설정 (반드시 import mujoco보다 먼저 해야 함!)
import os
os.environ['MUJOCO_GL'] = 'egl'

# 3. 라이브러리 설치
!pip install mujoco gymnasium stable_baselines3 imageio[ffmpeg]

# 4. 라이브러리 임포트 (이제 안전함)
import mujoco
import gymnasium as gym
from stable_baselines3 import PPO
import numpy as np
import imageio

print("설정 완료! 이제 에러가 안 날 겁니다.")


#역할: 프로젝트에 필요한 파이썬 라이브러리 4대장을 설치합니다.

#mujoco: 물리 엔진 (로봇이 움직이는 세상).

#gymnasium: 강화학습 표준 규격 (AI가 환경과 소통하는 인터페이스).

#stable_baselines3: PPO 알고리즘이 들어있는 두뇌 패키지.

#imageio: 눈에 안 보이는 시뮬레이션 화면을 **동영상 파일(mp4)**로 저장해 주는 촬영 기사.

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
!git clone https://github.com/google-deepmind/mujoco_menagerie.git
# 경로 확인: /content/mujoco_menagerie/unitree_go1/scene.xml

fatal: destination path 'mujoco_menagerie' already exists and is not an empty directory.


In [ ]:
import gymnasium as gym
import mujoco
import numpy as np
from gymnasium import spaces

class UnitreeGo1Env(gym.Env):
    def __init__(self):
        super().__init__()
        # 1. 모델 로드 xml파일을 읽어서 로봇의 물리 모델 생성
        self.model = mujoco.MjModel.from_xml_path("/content/mujoco_menagerie/unitree_go1/scene.xml")
        self.data = mujoco.MjData(self.model) #실제 움직이는 데이터는 여기 저장.

        # 2. Action Space 정의 (12개 관절 토크 제어) 다리(4)x개수(3)=12
        # 값의 범위는 -1 ~ 1 (나중에 스케일링)일단 범위는 작게하여 안정적으로
        self.action_space = spaces.Box(low=-1.0, high=1.0, shape=(12,), dtype=np.float32)

        # 3. Observation Space 정의 (관절 각도 + 속도 + 자이로 등) 30차원
        # 30개의 데이터를 갖고 PPO알고리즘이 직접 판단.
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(30,), dtype=np.float32)

    def reset(self, seed=None):
        mujoco.mj_resetData(self.model, self.data)
        # 초기 상태 반환
        return self._get_obs(), {}

    def step(self, action):
        # ... (이전 코드: action 적용 및 물리 스텝 진행) ...

        # -----------------------------------------------------------
        # [데이터 수집]
        # -----------------------------------------------------------
        # 1. 목표 속도 (예: 앞으로 1.0 m/s 가라)
        target_velocity = 1.0 
        
        # 2. 현재 상태 가져오기
        velocity_x = self.data.qvel[0] # 전진 속도
        base_ang_vel = self.data.qvel[3:6] # 몸통 회전 속도 (Roll, Pitch, Yaw)
        
        # 3. 관절 상태
        # qpos[7:]가 다리 관절입니다. (Go1은 다리당 3개 * 4 = 12개)
        # default_joint_pos는 로봇이 예쁘게 서 있는 자세의 각도값입니다.
        # (Go1의 경우 대략 [0, 0.9, -1.8] * 4 형태입니다. 값을 모르면 일단 0으로 둡니다)
        current_joint_pos = self.data.qpos[7:]
        default_joint_pos = np.array([0.0, 0.9, -1.8] * 4) 

        # -----------------------------------------------------------
        # [보상 함수 계산] (Reward Engineering)
        # -----------------------------------------------------------
        
        # 1. Tracking Reward (목표 속도 따라가기)
        # 속도 차이가 작을수록 점수가 높음 (Exponential kernel 사용 추천) -> 부드러운 보상곡선(급격한 변화 없음) 0~1값으로 안정적. 
        # -> gradient sensitvity -> 목표에 가까워질수록 점수가 급격히 증가
        v_err = target_velocity - velocity_x
        rew_tracking = np.exp(-5.0 * (v_err ** 2))

        # 2. Stability Reward (자세 제어)
        # 몸통 각속도(ang_vel)가 0에 가까워야 함 (흔들림 방지)
        rew_lin_vel_z = -np.square(self.data.qvel[2]) # 위아래로 방방 뛰지 마
        rew_ang_vel = -np.sum(np.square(base_ang_vel[:2])) # Roll, Pitch 흔들림 벌점
        ## Insights : penalty 는 제곱으로 벌점을 메기고 잘 한 건 gaussian 형태로 보상을 줌.

        # 3. Smoothness Reward (부드러운 움직임)
        # 관절이 기본 자세에서 얼마나 비틀어졌나?
        rew_dof_pos = -np.sum(np.square(current_joint_pos - default_joint_pos))
        # 토크를 얼마나 썼나?
        rew_torque = -np.sum(np.square(action)) 

        # -----------------------------------------------------------
        # [최종 합산] (가중치 튜닝이 실력!) -> 실제로 다른 연구들에서도 직접 튜닝한 가중치 사용 -> 노하우이자 실력
        # 이것에 대한 디테일한 내용은 추후에 더 찾아보면 좋을듯.
        # -----------------------------------------------------------
        total_reward = (
            1.0  * rew_tracking + 
            0.5  * rew_lin_vel_z + 
            0.05 * rew_ang_vel + 
            0.1  * rew_dof_pos + 
            0.005 * rew_torque
        )

        # 넘어짐 벌점
        terminated = False
        if self.data.qpos[2] < 0.2:
            terminated = True
            total_reward -= 10.0

        return self._get_obs(), total_reward, terminated, False, {}

    def _get_obs(self):

        # 현재 관절 각도와 속도를 합쳐서 리턴
        # MuJoCo 데이터 배열 구조:
        # qpos[0~2]: 몸통 위치 (x,y,z)
        # qpos[3~6]: 몸통 자세 (quaternion) -> 여기까지 7개는 '자유도' 정보
        # qpos[7~18]: 다리 관절 각도 (12개) -> 우리가 필요한 건 여기부터!
        
        # qvel[0~5]: 몸통 속도 (직선3 + 회전3)
        # qvel[6~17]: 다리 관절 속도 (12개) -> 우리가 필요한 건 여기부터!

        # 그래서 슬라이싱([7:], [6:])을 통해 몸통 정보는 빼고 "다리 상태"만 줍니다.
        # (참고: 보통 보행을 배우려면 몸통 기울기 정보도 주는 게 좋습니다)


        return np.concatenate([self.data.qpos[7:], self.data.qvel[6:]]).astype(np.float32)

    def close(self):
        pass

In [ ]:
from stable_baselines3 import PPO
import numpy as np
import types

# 환경 생성
env = UnitreeGo1Env()

# Fix: Monkey-patch the _get_obs method to return 30 elements
def _get_obs_fixed(self):
    return np.concatenate([
        self.data.qpos[7:],    # 12 joint positions
        self.data.qvel[0:3],   # 3 base linear velocities
        self.data.qvel[3:6],   # 3 base angular velocities
        self.data.qvel[6:]     # 12 joint velocities
    ]).astype(np.float32)

env._get_obs = types.MethodType(_get_obs_fixed, env)

# Fix: Downgrade sympy to a version compatible with torch/stable_baselines3
!pip install sympy==1.12

# PPO 모델 생성 (MlpPolicy: 이미지 말고 수치 데이터 사용)
model = PPO("MlpPolicy", env, verbose=1) # MLP = Multi-Layer Perceptron (완전 연결 신경망)을 사용
                                        # sensor data를 기반으로 학습하여 걷는 법을 배움. -> CNN, RNN이 아닌 MLP를 사용 
# verbose=1: 학습 진행 상황을 출력

# 학습 시작 (Colab 무료 버전 기준 10만~100만 스텝 추천)
model.learn(total_timesteps=500000) #50만 번의 행동을 하면서 배워라

# 모델 저장
model.save("ppo_unitree_go1")

  Using cached sympy-1.12-py3-none-any.whl.metadata (12 kB)
Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Attempting uninstall: sympy
    Found existing installation: sympy 1.14.0
    Uninstalling sympy-1.14.0:
      Successfully uninstalled sympy-1.14.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.9.0+cu126 requires sympy>=1.13.3, but you have sympy 1.12 which is incompatible.
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


Streaming output truncated to the last 5000 lines.
|    loss                 | 14.3        |
|    n_updates            | 170         |
|    policy_gradient_loss | -0.0619     |
|    std                  | 0.968       |
|    value_loss           | 29.8        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 37.7        |
|    ep_rew_mean          | -14.6       |
| time/                   |             |
|    fps                  | 423         |
|    iterations           | 19          |
|    time_elapsed         | 91          |
|    total_timesteps      | 38912       |
| train/                  |             |
|    approx_kl            | 0.023300787 |
|    clip_fraction        | 0.265       |
|    clip_range           | 0.2         |
|    entropy_loss         | -16.6       |
|    explained_variance   | 0.202       |
|    learning_rate        | 0.0003      |
|    loss                

In [ ]:
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install pyvirtualdisplay > /dev/null 2>&1

In [ ]:
import imageio
import numpy as np

from pyvirtualdisplay import Display

# 가상의 모니터를 켭니다 (보이진 않지만 백그라운드에서 돌아감)
display = Display(visible=0, size=(1400, 900))
display.start()

# 그 다음 import
import mujoco
# ...
# 1. 비디오 저장용 설정
video_path = "robot_result.mp4"
fps = 30
writer = imageio.get_writer(video_path, fps=fps)

# 2. 렌더러 초기화 (MuJoCo 화면 그리기 도구)
# 주의: env는 아까 만드신 UnitreeGo1Env 인스턴스여야 합니다.
renderer = mujoco.Renderer(env.model, height=480, width=640)

# 3. 테스트 시작
obs, _ = env.reset()
done = False
frame_count = 0
max_frames = 300  # 약 10초 분량만 녹화 (30fps * 10s)

print("비디오 녹화 시작... 🎬")

while not done and frame_count < max_frames:
    # 학습된 모델로 행동 결정 (deterministic=True는 '랜덤 없이 실력대로 해라'는 뜻)
    action, _ = model.predict(obs, deterministic=True)

    # 환경에 행동 적용
    obs, reward, terminated, truncated, _ = env.step(action)
    done = terminated or truncated

    # 화면 캡처 및 저장
    renderer.update_scene(env.data)
    pixels = renderer.render()
    writer.append_data(pixels)
    frame_count += 1

writer.close()
print(f"녹화 완료! '{video_path}' 파일이 생성되었습니다.")

비디오 녹화 시작... 🎬


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


녹화 완료! 'robot_result.mp4' 파일이 생성되었습니다.
